In [46]:
import mmsdk
import os
import argparse
import re
import csv
from mmsdk import mmdatasdk
from os import listdir
from os.path import isfile, join

In [ ]:
# args = ["All Labels.csd", "COVAREP.csd", "FACET 4.2", "glove_vectors.csd", "OpenFace_2.csd"]
args = "../final_aligned"
dataset_dictionary={}

if os.path.isdir(args) is False:
	print ("Folder does not exist ...")
	exit(-1)

csdfiles = [f for f in listdir(args) if isfile(join(args, f)) and f[-4:]=='.csd']
if len(csdfiles)==0:
	print("No csd files in the given folder")
	exit(-2)

print("%d csd files found"%len(csdfiles))
for csdfile in csdfiles:
	dataset_dictionary[csdfile]=os.path.join(args,csdfile)
dataset=mmdatasdk.mmdataset(dataset_dictionary)

print ("List of the computational sequences")
print (dataset.computational_sequences.keys())

In [74]:
# dimensions=["sentiment","happy","sad","anger","surprise","disgust","fear"]
# dataset["All Labels.csd"].data['r13W4H8ZhqI[0]']['features'][:] outputs the np.array(1,7) of the labels
# dataset["All Labels.csd"].data['r13W4H8ZhqI[0]']['features'].value works too
emotion_values = dataset["All Labels.csd"].data['-3g5yACwYnA[0]']['intervals'].value
print(emotion_values)

[[ 82.753 100.555]]


In [89]:
facetKeys = dataset['FACET 4.2.csd'].data['-3g5yACwYnA[0]']
print(facetKeys.keys())
facet = facetKeys["features"][:]
print(facet[0])
print(facet[-1])
# print(facet)
# Intervals are between [x - y] y is the time stamp that is related to the label. so [82.753 - 82.7667] means that 82.7667 is the key to 
# which csv row it relates to.

<KeysViewHDF5 ['features', 'intervals']>
[-1.47033   -0.631078  -0.930237  -2.27802   -1.31134    1.12753
 -0.128658  -1.6108     0.270326  -0.530749   0.21477   -0.163682
 -0.136293  -1.33055   -0.734238  -1.00315   -2.12545   -0.863226
 -1.24138   -1.27272   -1.04446   -0.790869  -0.312198   0.0379759
 -1.36377   -0.888425  -0.83577   -1.16119   -2.01143    2.69823
  0.0340883  0.957051   0.261945  -1.51155    2.58695  ]
[ 0.63548   -0.695839  -0.92402   -3.63815   -2.47539    0.586251
 -0.814577  -3.31504    1.45597    1.16186   -0.89285   -1.27751
  0.0808776 -1.06403   -0.735655  -0.708147  -1.29119   -0.437198
 -1.52764   -1.30229   -1.12694   -0.490785   0.349494  -0.237491
 -1.31652   -0.508255  -1.54333   -1.51823   -2.3163     1.57818
  0.0293242  0.985822  -1.12436    1.29823   -1.7477   ]


In [85]:
# The time intervals start on frame 2842 and end on 3016. Being off by 1 frame is also okay, should not affect anything
s = emotion_values[0][0]// 0.033333467
e = emotion_values[0][1]// 0.033333467
print(s,e)

2482.0 3016.0


In [95]:
label_data = dataset["All Labels.csd"]
labels_keys = dataset["All Labels.csd"].keys()
headers = ["video_name", "segmented_video", "segment_number", "start", "end", "start_frame", "end_frame", "sentiment", "happy", "sad", "anger",                    "surprise", "disgust", "fear"]
seconds_per_frame = 0.033333467
with open("mosei_labels.csv", "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(headers)
    count = 0
    for label in labels_keys:
        start_end = label_data.data[label]['intervals'][:]
        emotions = label_data.data[label]['features'][:]
        # start = start_end[0][0] + .0333467
        start = start_end[0][0]
        end = start_end[0][1]
        start_frame = int(start // seconds_per_frame)
        end_frame = int(end // seconds_per_frame)
        video_name = re.sub(r"\[.*\]", "", label)
        segment = re.findall(r"\[(.*?)\]", label)[0]
        segmented_video = f"{video_name}_{segment}"
        list_to_write = [label, segmented_video, segment, start, end, start_frame, end_frame, *emotions[0]]
        # print(list_to_write)
        csvwriter.writerow(list_to_write)
print("## Done!")

## Done!


In [96]:
import pandas as pd
hf=pd.read_csv("mosei_labels.csv") 
hf.head()

,video_name,segmented_video,segment_number,start,end,start_frame,end_frame,sentiment,happy,sad,anger,surprise,disgust,fear
0,--qXJuDtHPw[0],--qXJuDtHPw_0,0,23.199,30.325,695,909,1.000000,0.666667,0.000000,0.0,0.0,0.0,0.000000
1,-3g5yACwYnA[0],-3g5yACwYnA_0,0,82.753,100.555,2482,3016,1.000000,0.666667,0.666667,0.0,0.0,0.0,0.666667
2,-3g5yACwYnA[1],-3g5yACwYnA_1,1,119.919,125.299,3597,3758,0.666667,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,-3g5yACwYnA[2],-3g5yACwYnA_2,2,4.840,14.052,145,421,0.000000,0.666667,0.666667,0.0,0.0,0.0,0.333333
4,-3g5yACwYnA[3],-3g5yACwYnA_3,3,13.211,27.521,396,825,0.000000,0.333333,0.333333,0.0,0.0,0.0,0.000000
